# read s3 delta table

the following code can be used to read data from data lake table stored on s3 in the delta file format (similar to parquet). 

we will read the dataframe using pyspark.

In [32]:
import os
import ast
import logging
import pprint
import yaml
import json
import time
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.column import *
import datetime as dt
import pandas as pd
import numpy as np

import boto3
import pprint
import botocore.session
from botocore.exceptions import ClientError
from boto3.dynamodb.conditions import Key, Attr

with open("./../creds.json", "r") as g:
    creds = json.load(g)
    g.close()

pp = pprint.PrettyPrinter(indent = 1)
# print("creds.json keys: ")
# pp.pprint([k for k in creds.keys()])

spark_host = creds["spark_host"]
spark_port = creds["spark_port"]
aws_client = creds["aws-access-key"]
aws_secret = creds["aws-secret-access-key"]
extra_jar_list = creds["extra_jar_list"]
print("imported modules.")

s3 = boto3.client("s3")
print("connected to boto3 clients.")
print("spark host: {}".format(spark_host))
print("spark port: {}".format(spark_port))

imported modules.
connected to boto3 clients.
spark host: xanaxprincess.asuscomm.com
spark port: 7077


## delta table s3 path

In [33]:
bucket = "reddit-streaming-stevenhurwitt-new"
subreddit = "aws"
filepath = os.path.join("s3a://", bucket, subreddit + "_clean")
print(filepath)

s3a://reddit-streaming-stevenhurwitt-new/aws_clean


## spark

In [34]:
try:
    spark = SparkSession.builder.appName("twitter") \
        .master("spark://{}:{}".format(spark_host, spark_port)) \
        .config("spark.executor.memory", "2048m") \
        .config("spark.executor.cores", "2") \
        .config("spark.streaming.concurrentJobs", "8") \
        .config("spark.local.dir", "/opt/workspace/tmp/driver/") \
        .config("spark.worker.dir", "/opt/workspace/tmp/executor/") \
        .config("spark.eventLog.enabled", "true") \
        .config("spark.eventLog.dir", "/opt/workspace/tmp/events/") \
        .config("spark.sql.debug.maxToStringFields", 1000) \
        .config("spark.jars.packages", extra_jar_list) \
        .config("spark.hadoop.fs.s3a.access.key", aws_client) \
        .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
        .config("spark.hadoop.fs.s3a.buffer.dir", "/opt/workspace/tmp/blocks") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
        .enableHiveSupport() \
        .getOrCreate()

    sc = spark.sparkContext
    # index = 0
    sc.setLogLevel("WARN")
    # sc.setLocalProperty("spark.scheduler.pool", "pool{}".format(str(index)))
    print("imported modules, created spark.")

except Exception as f:
    print("EXCEPTION: ")

23/02/05 06:13:15 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServ

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

## read df

In [ ]:
try:
    df = spark.read.format("delta").option("header", True).load(filepath)
    df.show()

except Exception as g:
    print("EXCEPTION: {}".format(g))

EXCEPTION: name 'spark' is not defined


## pandas df

In [ ]:
df_pandas = df.toPandas()
df_pandas